## Import Libraries

In [1]:
import os
import time
import copy


# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt

from PIL import Image

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import timm

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm

cudnn.benchmark = True
plt.ion()   # interactive mode

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
device

device(type='cuda', index=0)

## Bring csv file with path & label

In [4]:
traindata_path = pd.read_csv('train_data.csv')
valdata_path = pd.read_csv('val_data.csv')
testdata_path = pd.read_csv('test_data.csv')

In [5]:
print(len(traindata_path))

15126


## Hyper Parameter
Resize([224,224]) - https://www.programcreek.com/python/example/104834/torchvision.transforms.Resize

In [6]:
transform = transforms.Compose([
#         transforms.Resize([224,224]),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
batch_size = 256
num_workers = 1

## Dataset

In [7]:
def readImage(path_data):
    path_to_image = []
    for path in path_data:
        img = Image.open(path)
        path_to_image.append(img)
    return path_to_image

In [8]:
class MyDataset(Dataset):
    def __init__(self, data, transform, train):
        self._repr_indent = 4
        self.data = data
        self.X = readImage(self.data['img_path'])
        self.y = self.data['label']
        self.transform = transform
        self.train = train
        self.classes = list(set(self.y))
    
    def __getitem__(self, idx):
        X, y = self.X[idx], None
#         X, y = self.X[idx], -1

        if self.transform:
            X = self.transform(X)
        if self.train:
            y = self.y[idx]

        return X,y
#         return torch.tensor(X), torch.tensor(y)
#         return X.clone().detach(), y.clone().detach()
    
    def __len__(self):
        return len(self.X)
    
    def __repr__(self):
        '''
        https://github.com/pytorch/vision/blob/master/torchvision/datasets/vision.py
        '''
        head = "My Custom Dataset : Mask Dataset"
        num_data = self._repr_indent*" " + "Number of datapoints: {}".format(self.__len__())
        num_classes = self._repr_indent*" " + "Number of classes: {}".format(len(self.classes))
        return '\n'.join([head, num_data, num_classes])

In [9]:
train_dataset = MyDataset(
    data = traindata_path,
    transform = transform,
    train = True
)
val_dataset = MyDataset(
    data = valdata_path,
    transform = transform,
    train = True
)
test_dataset = MyDataset(
    data = testdata_path,
    transform = transform,
    train = False
)

In [10]:
train_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 15126
    Number of classes: 18

In [11]:
val_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 3774
    Number of classes: 18

In [12]:
test_dataset

My Custom Dataset : Mask Dataset
    Number of datapoints: 12600
    Number of classes: 1

## DataLoader

In [13]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size = batch_size,
    num_workers = num_workers,
    shuffle = True,
    drop_last = False
)

In [14]:
dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [15]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f859f0ea580>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f859f0ea5e0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f859f0ea6d0>}

## Model
model 생성 및 확인 - https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055

In [16]:
model = timm.create_model('resnet18', pretrained=True)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 18)

In [17]:
model(torch.randn(64,3,384,512)).shape

torch.Size([64, 18])

In [18]:
count = 0 
for name, child in model.named_children():
    print(count,name)
    count+=1

count = 0

for param in model.children():
    if count < 9:
        param.requires_grad = False
    else:
        param.requires_grad = True

    count+=1

print()
count = 0

for param in model.children():
    print(count,param.requires_grad)
    count +=1


0 conv1
1 bn1
2 act1
3 maxpool
4 layer1
5 layer2
6 layer3
7 layer4
8 global_pool
9 fc

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 True


In [19]:
model = model.to(device)

loss_fn = torch.nn.CrossEntropyLoss() 

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) 

In [20]:
model.default_cfg

{'url': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
 'num_classes': 1000,
 'input_size': (3, 224, 224),
 'pool_size': (7, 7),
 'crop_pct': 0.875,
 'interpolation': 'bilinear',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'first_conv': 'conv1',
 'classifier': 'fc',
 'architecture': 'resnet18'}

In [21]:
model.get_classifier()

Linear(in_features=512, out_features=18, bias=True)

In [1]:
import torchsummary
torchsummary.summary(model,(3,224,224))

NameError: name 'model' is not defined

In [23]:
# model.state_dict()

## Train
transfer learning train model - https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html   

In [24]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [25]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()
    
#     best_model_wts = copy.deepcopy(model.state_dict()) 
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            running_loss = 0.
            running_acc = 0.
            if phase == "train":
                model.train() 
            elif phase == "val":
                model.eval() 

            for images, labels in dataloaders[phase]:
                images = images.to(device)
                labels = labels.to(device)

                optimizer.zero_grad() 

                with torch.set_grad_enabled(phase == "train"):
                    logits = model(images)
                    _, preds = torch.max(logits, 1)
#                     print(logits, labels)
                    loss = loss_fn(logits, labels)
#                     print("loss:",loss)

                    if phase == "train":
                        loss.backward() 
                        optimizer.step() 

                running_loss += loss.item() * images.size(0) 
                running_acc += torch.sum(preds == labels.data) 

            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_acc / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
#                 best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model

In [26]:
model = train_model(model, loss_fn, optimizer, num_epochs=25)

Epoch 0/24
----------
train Loss: 1.0204 Acc: 0.7279
val Loss: 0.5402 Acc: 0.8574

Epoch 1/24
----------
train Loss: 1.5310 Acc: 0.5170
val Loss: 1.6136 Acc: 0.4634

Epoch 2/24
----------
train Loss: 1.1914 Acc: 0.6069
val Loss: 1.4537 Acc: 0.5241

Epoch 3/24
----------
train Loss: 1.0330 Acc: 0.6550
val Loss: 1.4929 Acc: 0.4833

Epoch 4/24
----------
train Loss: 0.8739 Acc: 0.7012
val Loss: 1.4486 Acc: 0.5503

Epoch 5/24
----------
train Loss: 0.7379 Acc: 0.7462
val Loss: 1.5788 Acc: 0.4891

Epoch 6/24
----------
train Loss: 0.6087 Acc: 0.7892
val Loss: 1.6456 Acc: 0.5011

Epoch 7/24
----------
train Loss: 0.5168 Acc: 0.8269
val Loss: 1.7541 Acc: 0.4769

Epoch 8/24
----------
train Loss: 0.3874 Acc: 0.8814
val Loss: 1.9349 Acc: 0.4322

Epoch 9/24
----------
train Loss: 0.2944 Acc: 0.9172
val Loss: 2.1883 Acc: 0.4640

Epoch 10/24
----------
train Loss: 0.2610 Acc: 0.9312
val Loss: 1.8900 Acc: 0.4815

Epoch 11/24
----------
train Loss: 0.1507 Acc: 0.9686
val Loss: 2.1807 Acc: 0.4698

Ep

DataLoader worker (pid 4735) is killed by signal: Bus error. It is possible that dataloader's workers are out of shared memory. Please try to raise your shared memory limit.